In [1]:
from tensorly.decomposition import parafac
import torch
import numpy as np
import scipy
import tensorly as tl

In [2]:
tensor = torch.randn(3, 6, 6)
#print(tensor)
#u, s, v = np.linalg.svd(tensor)
#print(u)
#print(s)
#print(v.T)
tensor = np.array([[[1, 4, 5], [0, 4, 8], [2, 6, 6]],
                   [[2, 7, 8], [0, 6, 12], [4, 11, 10]],
                   [[1, 5, 7], [0, 6, 12], [2, 7, 8]]])
tensor = torch.from_numpy(tensor)
print(tensor)

tensor([[[ 1,  4,  5],
         [ 0,  4,  8],
         [ 2,  6,  6]],

        [[ 2,  7,  8],
         [ 0,  6, 12],
         [ 4, 11, 10]],

        [[ 1,  5,  7],
         [ 0,  6, 12],
         [ 2,  7,  8]]])


## Standard PARAFAC method from tensorly:

In [3]:
x = parafac(tl.tensor(tensor), rank=3)
print(x[0])
for i in x[1]:
    print(i)

[1 1 1]
[[ 1.39573578e+01 -2.63747243e+00  7.49884072e-17]
 [ 2.14232443e+01 -3.58290544e+00  3.23867218e-16]
 [ 2.04488291e+01 -4.32951185e+00 -9.89019967e-17]]
[[-0.46814108  0.15714387  4.95998065]
 [-0.62868266 -0.71714192 -1.43255439]
 [-0.62194084  0.6728587  10.63623848]]
[[-9.84937510e-02 -4.29697132e-01  4.79503916e+14]
 [-5.41393670e-01 -3.48248980e-01  8.55487044e+14]
 [-7.87306087e-01  5.92593438e-01  2.72462339e+14]]


## Implementation of ALS algorithm:

In [4]:
A = np.array([[1, 3], [2, 4]])
B = np.array([[5, 7], [6, 8]])
m = scipy.linalg.khatri_rao(A, B)
print(m)
print(A*B)

[[ 5 21]
 [ 6 24]
 [10 28]
 [12 32]]
[[ 5 21]
 [12 32]]


In [26]:
np.random.seed(1)
X_tensor = np.round(np.random.random((2, 3, 3))*10)
print(X_tensor)

[[[4. 7. 0.]
  [3. 1. 1.]
  [2. 3. 4.]]

 [[5. 4. 7.]
  [2. 9. 0.]
  [7. 4. 6.]]]


In [31]:
def compute_ALS_CANDECOMP_PARAFAC(input_matrix, max_iterations, rank):
    np.random.seed(1)
    A = np.round(np.random.randn(input_matrix.shape[-1], rank),2)
    while(max_iterations > 0):
        for i in range(0, input_matrix.shape[-1]):
            ## Step-01 (V calculation)
            A_new = A[:, 0:i+1]
            V = np.dot(A_new[:, 0:1], A_new[:, 0:1].T)
            if i != 0:
                for j in range(1, A_new.shape[-1]):
                    V = V*A_new[:, j:j+1]
            ## A Calculation
            V_inv = np.linalg.pinv(V)
            A_new = A[:, 0:i+1]
            A_KProduct = A_new[:, 0:1]
            if i != 0:
                for j in range(1, A_new.shape[-1]):
                    A_KProduct = scipy.linalg.khatri_rao(A_KProduct, A_new[:, j:j+1])
            X_unfolded = tl.unfold(input_matrix, mode = i)
            print("Shapes: ")
            print(X_unfolded.shape)
            print(A_KProduct.shape)
            print(V_inv.shape)
            print("######")
        max_iterations -= 1

In [32]:
compute_ALS_CANDECOMP_PARAFAC(X_tensor, 1, 3)

Shapes: 
(2, 9)
(3, 1)
(3, 3)
######
Shapes: 
(3, 6)
(9, 1)
(3, 3)
######
Shapes: 
(3, 6)
(27, 1)
(3, 3)
######
